In [8]:
import pandas as pd
import json

def get_sentiment(rating):
    if rating > 3:
        return 1
    elif rating < 3:
        return -1
    
    return None

def get_uri(movie_id):
    return None

def transform_imdb_id(imdb_id):
    return f'tt{str(imdb_id).zfill(7)}'


ratings = pd.read_csv('../data/ratings_movielens.csv')
print(f'{len(ratings)=}')

links = pd.read_csv(f'../data/links.csv')
mapping = pd.read_csv(f'../data/mapping.csv')

ratings = ratings.merge(links, on='movieId')
ratings.imdbId = ratings.imdbId.map(transform_imdb_id)
ratings = ratings.merge(mapping, on='imdbId')

ratings['sentiment'] = ratings.rating.transform(get_sentiment)
# ratings['uri'] = ratings.movieId.transform(get_uri)
ratings['isItem'] = True
ratings.drop(columns=['rating', 'timestamp', 'tmdbId', 'imdbId', 'movieId'], inplace=True)
ratings.dropna(inplace=True)
print(f'{len(ratings)=}')

entities = pd.read_csv('../data_loading/mindreader/entities.csv')
ratings = ratings.merge(entities, on='uri')
print(f'{len(ratings)=}')

with open('../data/ml_transformed.csv', 'w') as fp:
    fp.write(ratings.to_csv(index=False))

rating_list = list()

for index, row in ratings.iterrows():
    rating_list.append((str(row.userId), row.uri, row.sentiment))

with open('../data/ml_ratings.json', 'w') as fp:
    json.dump(rating_list, fp)

len(ratings)=100836
len(ratings)=80253
len(ratings)=75630
